# VacationPy
----


In [51]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [52]:

# Create a reference the CSV file desired
csv_path = "output_data/cities.csv"

# Read the CSV into a Pandas DataFrame
cities_df= pd.read_csv(csv_path)

# Print the first five rows of data to the screen
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Talaya,61.3833,152.7500,54.61,95,100,2.30,RU,1627081705
1,Vaini,-21.2000,-175.2000,80.76,83,40,8.05,TO,1627081705
2,Avarua,-21.2078,-159.7750,78.85,65,3,13.80,CK,1627081706
3,Novyy Svit,47.8059,38.0211,64.33,56,0,5.35,UA,1627081706
4,Pochutla,15.7432,-96.4661,89.82,61,60,6.24,MX,1627081706


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [53]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = cities_df[["Lat", "Lng"]]
weight = cities_df["Humidity"]

In [54]:
#Add Heatmap layer to map.
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights = weight, 
                                 dissipating=False, max_intensity=np.max(weight),
                                 point_radius=3)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [43]:
#   * A max temperature lower than 80 degrees but higher than 70.

#   * Wind speed less than 10 mph.

#   * Zero cloudiness.

#   * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
# Dropped cities with missing values
cities_new_df = cities_df.dropna()
cities_new_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Talaya,61.3833,152.7500,54.61,95,100,2.30,RU,1627081705
1,Vaini,-21.2000,-175.2000,80.76,83,40,8.05,TO,1627081705
2,Avarua,-21.2078,-159.7750,78.85,65,3,13.80,CK,1627081706
3,Novyy Svit,47.8059,38.0211,64.33,56,0,5.35,UA,1627081706
4,Pochutla,15.7432,-96.4661,89.82,61,60,6.24,MX,1627081706


In [44]:
#   * A max temperature lower than 80 degrees but higher than 70.

#   * Wind speed less than 10 mph.

#   * Zero cloudiness.
filtered_cities = cities_new_df.loc[(cities_new_df["Max Temp"] <= 80) & 
                                    (cities_new_df["Max Temp"] >= 70) & 
                                    (cities_new_df["Wind Speed"] < 10) &  
                                    (cities_new_df["Cloudiness"] == 0)]
                                
filtered_cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,Piranshahr,36.7010,45.1413,70.95,43,0,2.33,IR,1627081707
58,Quetigny,47.3167,5.1167,78.53,83,0,6.91,FR,1627081725
117,Algiers,36.7525,3.0420,75.02,83,0,2.30,DZ,1627081623
183,Aksu,41.1231,80.2644,76.17,51,0,5.64,CN,1627081765
244,Pano Polemidya,34.7000,33.0000,79.72,89,0,8.05,CY,1627081786
246,Mizdah,31.4451,12.9801,73.09,55,0,9.06,LY,1627081786
285,Ürümqi,43.8010,87.6005,76.15,41,0,6.71,CN,1627081799
345,Ushtobe,45.2514,77.9838,74.12,30,0,5.01,KZ,1627081823
349,Coxim,-18.5067,-54.7600,79.47,28,0,4.14,BR,1627081696
361,Crotone,39.0851,17.1178,71.92,60,0,4.88,IT,1627081682


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [45]:
hotel_df = pd.DataFrame(filtered_cities)
hotel_df["Hotel Name"] = ""

In [46]:
# geocoordinates
target_search = "hotels"
#Set parameters to search for hotels with 5000 meters.
target_radius = 5000
target_type = "Business"

# params dictionary to update each iteration
params = {
    "keyword": target_search,
    "radius":target_radius,
    "type":target_type,
    "key" : g_key
}
    
# Use the lat/lng we recovered to identify 
for index, row in hotel_df.iterrows():
# Use the search term: "hotels" and our lat/ln
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # make request and print url
    hotel_info = requests.get(base_url, params=params)
    hotel_info = hotel_info.json()
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_info["results"][0]["name"]
        hotel_df.loc[index, "Hotel Address"] = hotel_info["results"][0]["vicinity"]
        hotel_df.loc[index, "Hotel Rating"] = hotel_info["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
       

Missing field/result... skipping.


In [47]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Address,Hotel Rating
5,Piranshahr,36.7010,45.1413,70.95,43,0,2.33,IR,1627081707,Hotel Ronak,Piranshahr,2.7
58,Quetigny,47.3167,5.1167,78.53,83,0,6.91,FR,1627081725,Hôtel Mercure Dijon Centre Clemenceau,"22 Bd de la Marne, Dijon",4.3
117,Algiers,36.7525,3.0420,75.02,83,0,2.30,DZ,1627081623,Hotel Aurassi,"Hotel El-Aurassi, 2 Ave du Docteur Frantz Fano...",4.2
183,Aksu,41.1231,80.2644,76.17,51,0,5.64,CN,1627081765,Tianyuan International Hotel,阿克苏市滨河路10号,5.0
244,Pano Polemidya,34.7000,33.0000,79.72,89,0,8.05,CY,1627081786,S Paul Hotel,"5, Ifigeneias Str, Limassol",4.7
246,Mizdah,31.4451,12.9801,73.09,55,0,9.06,LY,1627081786,فندق مزدة الكبير,Mizdah,1.5
285,Ürümqi,43.8010,87.6005,76.15,41,0,6.71,CN,1627081799,Sheraton Urumqi Hotel,"669 Northern Youhao Road, Ürümqi",3.9
345,Ushtobe,45.2514,77.9838,74.12,30,0,5.01,KZ,1627081823,,NaN,NaN
349,Coxim,-18.5067,-54.7600,79.47,28,0,4.14,BR,1627081696,Hotel Avenida,"Av. Virgínia Ferreira, 465 - São Judas Tadeu, ...",3.4
361,Crotone,39.0851,17.1178,71.92,60,0,4.88,IT,1627081682,Hotel San Giorgio,"S.da Statale 106 Jonica, Crotone",4.3


In [48]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [49]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content= hotel_info)
# Display figure
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))

In [50]:
#Plotting the hotels on top of the humidity heatmap with each pin containing the **Hotel Name**, **City**, and **Country**.fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

fig.add_layer(heat_layer)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))